In [888]:
from sympy import symbols, sqrt, exp, log, ccode, solve, Function
elastic, eta_el, f_ani     = symbols('elastic, eta_el, f_ani')
Ag, lam, gam, cg, Tii, pg  = symbols('Ag, lambda, gamma, c, Tii, p')
Eii_pwl, Eii_exp, Eii_gbs, Eii_pl, Eii = symbols('Eii_pwl, Eii_exp, Eii_gbs, Eii_pl, Eii')
C_lin, n_lin, m_lin, C_pwl, n_pwl = symbols('C_lin, n_lin, m_lin, C_pwl, n_pwl')
d1, eta_ve = symbols('d1, eta_ve')
Bg    = symbols('Bg')

In [889]:
# Dissipation
Exx, Eyy, Exy  = symbols('Exx, Eyy, Exy')
Txx  = 2*eta_ve*Exx
Tyy  = 2*eta_ve*Eyy
Txy  = 2*eta_ve*Exy
W    = Txx*Exx + Tyy*Eyy + 2*Txy*Exy
Eii2 = 1/2*(Exx**2 + Eyy**2) + Exy**2
Tii2 = 1/2*(Txx**2 + Tyy**2) + Txy**2
(W - (4*eta_ve*Eii2)).simplify()  

0

In [890]:
# Dissipation
Exx, Eyy, Exy  = symbols('Exx, Eyy, Exy')
Exx_pwl, Eyy_pwl, Exy_pwl  = symbols('Exx_pwl, Eyy_pwl, Exy_pwl')
Txx  = 2*eta_ve*Exx
Tyy  = 2*eta_ve*Eyy
Txy  = 2*eta_ve*Exy
W    = Txx*Exx_pwl + Tyy*Eyy_pwl + 2*Txy*Exy_pwl
W1   = 4*eta_ve*(1/2*(Exx*Exx_pwl + Eyy*Eyy_pwl) + Exy*Exy_pwl)
display((W - W1).simplify()) 
# Using normality rule
Exx_pwl = Txx*Eii_pwl/Tii
Eyy_pwl = Tyy*Eii_pwl/Tii
Exy_pwl = Txy*Eii_pwl/Tii
W       = Txx*Exx_pwl + Tyy*Eyy_pwl + 2*Txy*Exy_pwl
W1      = 2*Eii_pwl*Tii2/Tii
# W1   = 2*Eii_pwl*Tii <----------------
display((W - W1).simplify())  #.subs(Tii, sqrt(Tii2))

0

0

In [891]:
display(4*eta_ve**2*Eii2)
display(Tii2)
display((4*eta_ve**2*Eii2- Tii2).simplify())

4*eta_ve**2*(0.5*Exx**2 + Exy**2 + 0.5*Eyy**2)

2.0*Exx**2*eta_ve**2 + 4*Exy**2*eta_ve**2 + 2.0*Eyy**2*eta_ve**2

0

In [892]:
# Solve for steady state grain size
Tii   = 2*eta_ve*Eii
W_pwl = Tii * Eii_pwl 
W_pwl = 4 * eta_ve * Eii_pwl**2
dddt  = 0
beta  = 1
Bg    = symbols('Bg')
# Bg   = beta*lam/cg/gam
d     = symbols('d')  
F     = dddt - (Bg * W_pwl *d**2) + Ag/pg*d**(1-pg)
d2    = solve(F,d)[0]
print(d2)
display(d2)

(4*Bg*Eii_pwl**2*eta_ve*p/Ag)**(-1/(p + 1))


(4*Bg*Eii_pwl**2*eta_ve*p/Ag)**(-1/(p + 1))

In [893]:
# Single residual version
d3          = d2.subs(Eii_pwl, C_pwl * pow(Tii, n_pwl))
d4          = Function('d')(eta_ve)
Eii_pwl     = C_pwl * pow(Tii, n_pwl    )
Eii_lin     = C_lin * pow(Tii, n_lin) * pow(d4,-m_lin)
Eii_vis     = Eii_pwl + Eii_lin + Eii_exp + Eii_gbs
r_eta_ve    = Eii - elastic*Tii/(2.0*eta_el) - Eii_vis
dr_deta     = r_eta_ve.diff(eta_ve) 
print('dddeta      = ' + ccode(d3.diff(eta_ve).subs(d3, 'd')) + ';')
print('dr_eta_deta = ' + ccode(dr_deta.subs(Eii_lin,'Eii_lin').subs(Eii_pwl,'Eii_pwl').subs(d4.diff(eta_ve), 'dddeta').subs(d4,'d')) + ';' )

display(d3.diff(eta_ve).subs(d3, 'd'))
display(dr_deta.subs(Eii_lin,'Eii_lin').subs(Eii_pwl,'Eii_pwl').subs(d3.diff(eta_ve), 'dddeta'))


dddeta      = -1.0/4.0*Ag*d*pow(2*Eii*eta_ve, -2*n_pwl)*(8*Bg*pow(C_pwl, 2)*n_pwl*p*pow(2*Eii*eta_ve, 2*n_pwl)/Ag + 4*Bg*pow(C_pwl, 2)*p*pow(2*Eii*eta_ve, 2*n_pwl)/Ag)/(Bg*pow(C_pwl, 2)*eta_ve*p*(p + 1));
dr_eta_deta = -1.0*Eii*elastic/eta_el - Eii_lin*n_lin/eta_ve + Eii_lin*dddeta*m_lin/d - Eii_pwl*n_pwl/eta_ve;


-Ag*d*(8*Bg*C_pwl**2*n_pwl*p*(2*Eii*eta_ve)**(2*n_pwl)/Ag + 4*Bg*C_pwl**2*p*(2*Eii*eta_ve)**(2*n_pwl)/Ag)/(4*Bg*C_pwl**2*eta_ve*p*(2*Eii*eta_ve)**(2*n_pwl)*(p + 1))

-1.0*Eii*elastic/eta_el + Eii_lin*m_lin*Derivative(d(eta_ve), eta_ve)/d(eta_ve) - Eii_lin*n_lin/eta_ve - Eii_pwl*n_pwl/eta_ve

In [894]:
# Define the two residuals
Tii      = 2*eta_ve*Eii
Eii_pwl  = C_pwl * pow(Tii, n_pwl    )
Eii_lin  = C_lin * pow(Tii, n_lin) * pow(d1,-m_lin)
d_it     = exp(log( Ag *gam/(lam*(1.0/cg)* Tii * Eii_pwl*pg))/(1.0+pg))
Eii_vis  = Eii_pwl + Eii_lin + Eii_exp + Eii_gbs
r_eta_ve = Eii - elastic*Tii/(2.0*eta_el) - Eii_vis
r_d      = d1 - d_it

In [895]:
# Jacobian
dr_eta_deta = r_eta_ve.diff(eta_ve) 
dr_eta_dd   = r_eta_ve.diff(d1) 
dr_d_deta   = r_d.diff(eta_ve) 
dr_d_dd     = r_d.diff(d1)
print('dr_eta_deta = ' +  ccode(dr_eta_deta.subs(Eii_lin,'Eii_lin').subs(Eii_pwl,'Eii_pwl')) + ';' )
print('dr_eta_dd   = ' +  ccode(dr_eta_dd.subs(Eii_lin,'Eii_lin')) + ';' )
print('dr_d_deta   = ' +  ccode(dr_d_deta.subs(d_it,'d_it').subs(Eii_pwl,'Eii_pwl').subs(Eii_lin,'Eii_lin')) + ';' )
print('dr_d_dd     = ' +  ccode(dr_d_dd.subs(Eii_lin,'Eii_lin')) + ';' )

dr_eta_deta = -1.0*Eii*elastic/eta_el - Eii_lin*n_lin/eta_ve - Eii_pwl*n_pwl/eta_ve;
dr_eta_dd   = Eii_lin*m_lin/d1;
dr_d_deta   = -2.0*Eii*Eii_pwl*d_it*eta_ve*lambda*p*(-0.5*Ag*c*gamma*n_pwl/(Eii*Eii_pwl*pow(eta_ve, 2)*lambda*p) - 0.5*Ag*c*gamma/(Eii*Eii_pwl*pow(eta_ve, 2)*lambda*p))/(Ag*c*gamma*(p + 1.0));
dr_d_dd     = 1;
